# 05 - Maintenance du Modèle

Ce notebook présente les outils de MLOps pour la maintenance du modèle de segmentation :

1. **Model Registry** : Versioning des modèles avec métadonnées
2. **Drift Detection** : Détection de drift de données et de modèle
3. **Simulation de réentraînement** : Estimation de la fréquence optimale

---

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.loader import load_transactions
from src.features.rfm import RFMCalculator
from src.models.clustering import CustomerSegmenter
from src.models.evaluation import evaluate_clustering
from src.monitoring import ModelRegistry, DriftDetector, calculate_ari
from src.monitoring.drift import estimate_retraining_frequency

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

print("Modules chargés avec succès!")

## 1. Chargement des données et du modèle

Commençons par charger les données RFM et entraîner un modèle de référence.

In [ ]:
# Charger les données
try:
    rfm_df = pd.read_parquet('../data/processed/customers_rfm.parquet')
    print(f"Données RFM chargées: {len(rfm_df)} clients")
except FileNotFoundError:
    print("Calcul des features RFM...")
    df = load_transactions('../data/raw/data.csv')
    calculator = RFMCalculator(reference_date='2018-09-01')
    rfm_df = calculator.fit_transform(df)
    print(f"RFM calculé: {len(rfm_df)} clients")

rfm_df.head()

In [ ]:
# Entraîner le modèle de référence
segmenter = CustomerSegmenter(n_clusters=4, random_state=42)
labels = segmenter.fit_predict(rfm_df)

# Évaluer
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df)

metrics = evaluate_clustering(rfm_scaled, labels)
print(f"\nMétriques du modèle:")
for key, value in metrics.items():
    print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")

## 2. Model Registry

Le Model Registry permet de versionner les modèles avec leurs métadonnées.

In [ ]:
# Initialiser le registre
registry = ModelRegistry()

# Calculer le hash des données
data_hash = ModelRegistry.compute_data_hash(rfm_df)
print(f"Hash des données: {data_hash}")

In [ ]:
# Enregistrer le modèle dans le registre
version = registry.register(
    model=segmenter.model,
    scaler=segmenter.scaler,
    metrics={
        'silhouette': metrics['silhouette'],
        'calinski_harabasz': metrics['calinski_harabasz'],
        'davies_bouldin': metrics['davies_bouldin'],
    },
    hyperparameters={
        'n_clusters': 4,
        'random_state': 42,
        'n_init': 10,
        'max_iter': 300,
    },
    data_hash=data_hash,
    n_samples=len(rfm_df),
    description="Modèle initial de segmentation RFM",
)

print(f"\nModèle enregistré: v{version}")

In [ ]:
# Lister les versions
versions_df = registry.list_versions()
print("Versions enregistrées:")
versions_df

In [ ]:
# Charger un modèle depuis le registre
loaded_model, loaded_scaler, loaded_meta = registry.load()

print(f"Modèle chargé: v{loaded_meta['version']}")
print(f"Créé le: {loaded_meta['created_at']}")
print(f"Métriques: {loaded_meta['metrics']}")

## 3. Drift Detection

La détection de drift permet d'identifier quand le modèle doit être réentraîné.

In [ ]:
# Initialiser le détecteur de drift
detector = DriftDetector(ari_threshold=0.8, pvalue_threshold=0.05)

# Configurer avec les données de référence
detector.fit(rfm_df, labels)

print("Détecteur configuré avec les données de référence")

In [ ]:
# Simuler des nouvelles données avec un léger drift
np.random.seed(42)

# Simuler un drift de 10% sur les features
new_data = rfm_df.copy()
for col in new_data.columns:
    noise = np.random.normal(0, new_data[col].std() * 0.1, len(new_data))
    new_data[col] = new_data[col] + noise

print("Données simulées avec drift de 10%")
print(f"\nComparaison des moyennes:")
comparison = pd.DataFrame({
    'Reference': rfm_df.mean(),
    'Current': new_data.mean(),
    'Diff (%)': ((new_data.mean() - rfm_df.mean()) / rfm_df.mean() * 100).round(2)
})
comparison

In [ ]:
# Détecter le drift
report = detector.detect(
    current_data=new_data,
    model=segmenter.model,
    scaler=segmenter.scaler,
)

print("=" * 60)
print("RAPPORT DE DRIFT")
print("=" * 60)
print(f"\nDate: {report.timestamp}")
print(f"\nDrift de données détecté: {report.data_drift_detected}")
print(f"Drift de modèle détecté: {report.model_drift_detected}")
print(f"Score ARI: {report.ari_score:.4f}")
print(f"\nDétails par feature:")
for feat, details in report.feature_drifts.items():
    status = "⚠️ DRIFT" if details['drift_detected'] else "✓ OK"
    print(f"  {feat}: {status} (p-value: {details['pvalue']:.4f}, shift: {details['mean_shift_percent']:.1f}%)")

print(f"\n📋 RECOMMANDATION:")
print(f"   {report.recommendation}")

## 4. Simulation de la fréquence de réentraînement

Simulons l'évolution du drift dans le temps pour estimer la fréquence optimale de réentraînement.

In [ ]:
# Simulation temporelle
simulation = detector.simulate_temporal_drift(
    data=rfm_df,
    model=segmenter.model,
    scaler=segmenter.scaler,
    n_periods=12,  # 12 mois
    drift_rate=0.05,  # 5% de drift par mois
)

print("Simulation du drift sur 12 mois:")
simulation

In [ ]:
# Visualiser l'évolution de l'ARI
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(simulation['period'], simulation['ari_score'], 'b-o', linewidth=2, markersize=8)
ax.axhline(y=0.8, color='r', linestyle='--', label='Seuil de réentraînement (ARI=0.8)')

# Marquer la zone de réentraînement
ax.fill_between(simulation['period'], 0, 0.8, alpha=0.2, color='red', label='Zone de réentraînement nécessaire')

ax.set_xlabel('Période (mois)', fontsize=12)
ax.set_ylabel('Score ARI', fontsize=12)
ax.set_title('Évolution de la stabilité du modèle dans le temps', fontsize=14)
ax.legend(loc='lower left')
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Estimer la fréquence optimale
estimation = estimate_retraining_frequency(
    data=rfm_df,
    model=segmenter.model,
    scaler=segmenter.scaler,
    ari_threshold=0.8,
    drift_rate=0.05,  # 5% de drift mensuel estimé
)

print("=" * 60)
print("ESTIMATION DE LA FRÉQUENCE DE RÉENTRAÎNEMENT")
print("=" * 60)
print(f"\nPériodes avant drift critique: {estimation['periods_until_drift']} mois")
print(f"ARI au moment du drift: {estimation['ari_at_drift']:.4f}")
print(f"\n📋 {estimation['recommendation']}")

## 5. Workflow de réentraînement

Exemple de workflow complet pour réentraîner le modèle quand un drift est détecté.

In [ ]:
def retrain_if_needed(
    current_data: pd.DataFrame,
    detector: DriftDetector,
    registry: ModelRegistry,
    segmenter: CustomerSegmenter,
) -> dict:
    """
    Workflow de réentraînement conditionnel.
    
    Returns
    -------
    dict
        Résultat du workflow avec action prise.
    """
    # 1. Détecter le drift
    report = detector.detect(
        current_data=current_data,
        model=segmenter.model,
        scaler=segmenter.scaler,
    )
    
    result = {
        'drift_report': report.to_dict(),
        'action_taken': None,
        'new_version': None,
    }
    
    # 2. Décider de l'action
    if report.model_drift_detected:
        print("🔄 Drift de modèle détecté - Réentraînement en cours...")
        
        # Réentraîner
        new_segmenter = CustomerSegmenter(n_clusters=4, random_state=42)
        new_labels = new_segmenter.fit_predict(current_data)
        
        # Évaluer
        new_scaled = new_segmenter.scaler.transform(current_data)
        new_metrics = evaluate_clustering(new_scaled, new_labels)
        
        # Enregistrer la nouvelle version
        new_version = registry.register(
            model=new_segmenter.model,
            scaler=new_segmenter.scaler,
            metrics={
                'silhouette': new_metrics['silhouette'],
                'calinski_harabasz': new_metrics['calinski_harabasz'],
                'davies_bouldin': new_metrics['davies_bouldin'],
            },
            hyperparameters={'n_clusters': 4, 'random_state': 42},
            data_hash=ModelRegistry.compute_data_hash(current_data),
            n_samples=len(current_data),
            description=f"Réentraînement suite au drift (ARI={report.ari_score:.3f})",
            bump='minor',
        )
        
        # Mettre à jour le détecteur
        detector.fit(current_data, new_labels)
        
        result['action_taken'] = 'retrained'
        result['new_version'] = new_version
        result['new_metrics'] = new_metrics
        
        print(f"✅ Nouveau modèle enregistré: v{new_version}")
        print(f"   Silhouette: {new_metrics['silhouette']:.4f}")
        
    elif report.data_drift_detected:
        print("📊 Drift de données détecté - Surveillance renforcée")
        result['action_taken'] = 'monitoring'
        
    else:
        print("✓ Aucun drift détecté - Modèle stable")
        result['action_taken'] = 'none'
    
    return result

In [ ]:
# Tester le workflow avec des données fortement driftées
np.random.seed(123)
heavily_drifted_data = rfm_df.copy()
for col in heavily_drifted_data.columns:
    noise = np.random.normal(0, heavily_drifted_data[col].std() * 0.3, len(heavily_drifted_data))
    heavily_drifted_data[col] = heavily_drifted_data[col] + noise

print("Test avec données fortement driftées (30%):\n")
result = retrain_if_needed(
    current_data=heavily_drifted_data,
    detector=detector,
    registry=registry,
    segmenter=segmenter,
)

In [ ]:
# Voir toutes les versions après réentraînement
print("\nHistorique des versions:")
registry.list_versions()

## 6. Résumé et recommandations

### Conclusions

1. **Fréquence de réentraînement recommandée** : Tous les 3-4 mois
2. **Seuil ARI critique** : 0.8 (en dessous = réentraînement nécessaire)
3. **Métriques à surveiller** : 
   - Score Silhouette (stabilité des clusters)
   - Distribution des features RFM
   - Score ARI par rapport au modèle de référence

### Actions recommandées

| Situation | Action |
|-----------|--------|
| ARI > 0.9 | Aucune action, check dans 1 mois |
| 0.8 < ARI < 0.9 | Surveillance renforcée |
| ARI < 0.8 | Réentraînement recommandé |
| Drift données + ARI < 0.8 | Réentraînement urgent |

In [ ]:
print("Notebook de maintenance terminé!")
print("\nPour automatiser ce workflow, consultez le GitHub Action dans .github/workflows/maintenance.yml")